<a href="https://colab.research.google.com/github/awjans/CopilotForPRsAdoption/blob/main/scripts/AIDev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Collection/Cleaning Overview
1. **PR identification**
   * Queried GitHub via GraphQL for PRs whose description contained the phrase **“Generated by Copilot”** or any of the marker tags:

     * `copilot:summary`
     * `copilot:walkthrough`
     * `copilot:poem`
     * `copilot:all`

2. **Scope**
   * Collected **18,256 PRs** from **146 early-adopter repositories** during **March 2023 – August 2023**.

3. **Control set**
   * For the same repositories, gathered **54,188 PRs** that did **not** contain any Copilot marker.
   * These served as the **untreated (control) group** for the **RQ2 comparison**.

4. **Bot filtering**
   * Removed PRs and comments authored by bots using the **high-precision method** of **Golzadeh et al. (2022)**, which included:
     * (i) Usernames ending with “bot”
     * (ii) A curated list of **527 known bot accounts**

5. **Revision extraction (RQ3)**
   * From the **18,256 Copilot-generated PRs**, retrieved the full **edit history** of PR descriptions.
   * Identified **1,437 revisions** where developers **edited the AI-suggested content**.

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
import asyncio
import matplotlib.pyplot as plt
import nest_asyncio
import numpy as np
import os
import pandas as pd
import requests
import seaborn as sns
import datetime
from dateutil import parser
from google.colab import userdata
from urllib.parse import urlparse
import time
import random
from itertools import cycle
from google.colab import userdata
from dateutil import parser
from urllib.parse import urlparse

# **First**, Define the URLs of the AIDev Parquet Files that we are intersted in.

In [23]:
pull_request_file_url = 'https://huggingface.co/datasets/hao-li/AIDev/resolve/main/all_pull_request.parquet'
pr_comments_file_url = 'https://huggingface.co/datasets/hao-li/AIDev/resolve/main/pr_comments.parquet'
pr_commits_file_url = 'https://huggingface.co/datasets/hao-li/AIDev/resolve/main/pr_commits.parquet'
pr_commit_details_file_url = 'https://huggingface.co/datasets/hao-li/AIDev/resolve/main/pr_commit_details.parquet'
pr_reviews_file_url = 'https://huggingface.co/datasets/hao-li/AIDev/resolve/main/pr_reviews.parquet'
pr_review_comments_file_url = 'https://huggingface.co/datasets/hao-li/AIDev/resolve/main/pr_review_comments.parquet'
pr_task_type_file_url = 'https://huggingface.co/datasets/hao-li/AIDev/resolve/main/pr_task_type.parquet'
repository_file_url = 'https://huggingface.co/datasets/hao-li/AIDev/resolve/main/all_repository.parquet'
user_file_url = 'https://huggingface.co/datasets/hao-li/AIDev/resolve/main/user.parquet'


# **Second**, We need to load the data from the URLs (15s)

### Load Parquet Files

In [24]:
"""
Load the Parquet file into a Pandas DataFrame from the file URL.
"""
def load_data(url: str):
  import pandas as pd # Import pandas inside the function
  try:
    # For Parquet files:
    df = pd.read_parquet(url)

    return df
  except Exception as e:
      print(f"Error loading data: {e}")
      print("Please ensure the URL is correct and the file is publicly accessible.")
      return None # Return None in case of an error

In [25]:
pull_request = load_data(pull_request_file_url)
pr_comments = load_data(pr_comments_file_url)
pr_commits = load_data(pr_commits_file_url)
pr_commit_details = load_data(pr_commit_details_file_url)
pr_reviews = load_data(pr_reviews_file_url)
pr_review_comments = load_data(pr_review_comments_file_url)
pr_task_type = load_data(pr_task_type_file_url)
repository = load_data(repository_file_url)
user = load_data(user_file_url)

In [26]:
display(pr_reviews.head(25))

,id,pr_id,user,user_type,state,submitted_at,body
0,2885691382,3107321792,coderabbitai[bot],Bot,COMMENTED,2025-06-01T14:22:22Z,**Actionable comments posted: 2**\n\n<details>...
1,2885712797,3107321792,coderabbitai[bot],Bot,COMMENTED,2025-06-01T14:37:45Z,**Actionable comments posted: 1**\n\n<details>...
2,3059587397,3234660269,Fank,User,COMMENTED,2025-07-27T15:06:40Z,None
3,3059589121,3234660269,benfdking,User,COMMENTED,2025-07-27T15:12:52Z,None
4,2813254905,3037457814,wilsonccccc,User,APPROVED,2025-05-03T15:34:42Z,None
5,2679908484,2915198291,ellipsis-dev[bot],Bot,COMMENTED,2025-03-12T21:24:55Z,:+1: Looks good to me! Reviewed everything up ...
6,3011938086,3223924413,davinchia,User,COMMENTED,2025-07-11T20:48:38Z,None
7,3034262238,3223924413,aaronsteers,User,APPROVED,2025-07-18T17:11:06Z,None
8,2866389127,3088611576,copilot-pull-request-reviewer[bot],Bot,COMMENTED,2025-05-24T17:14:22Z,## Pull Request Overview\n\nImplements dot‐not...
9,2866393218,3088611576,parruda,User,APPROVED,2025-05-24T17:34:15Z,None


### Github token cycling & safe_get functions

In [27]:
nest_asyncio.apply()

GITHUB_TOKENS = [
    os.environ.get("GITHUB_TOKEN"),
    userdata.get('GITHUB_TOKEN_2'),
    userdata.get('GITHUB_TOKEN_3'),
    userdata.get('GITHUB_TOKEN_4'),
    userdata.get('GITHUB_TOKEN_5'),
    userdata.get('GITHUB_TOKEN_6'),
    userdata.get('GITHUB_TOKEN_7'),
]
# Filter out any None values if a token is not set
GITHUB_TOKENS = [token for token in GITHUB_TOKENS if token is not None]

if not GITHUB_TOKENS:
    raise RuntimeError("❌ No GitHub tokens found. Please add your tokens to Colab Secrets.")

# Create a cycle iterator for the tokens
token_cycle = cycle(GITHUB_TOKENS)

def safe_get(url, sleep_base=1.0, retries=3):
    """GET with retry, error handling, and rate-limit backoff, cycling through tokens."""
    current_token = next(token_cycle) # Get the next token in the cycle
    headers = {'Authorization': f'token {current_token}', # Use the current token
               'Accept': 'application/vnd.github+json'}
    for attempt in range(retries):
        try:
            r = requests.get(url, headers=headers)
            # --- Rate limit exceeded ---
            if r.status_code == 403 and 'X-RateLimit-Remaining' in r.headers and r.headers['X-RateLimit-Remaining'] == '0':
                reset_time = int(r.headers.get('X-RateLimit-Reset', time.time()+60))
                sleep_for = max(reset_time - time.time(), 30)
                print(f"[RATE-LIMIT] Sleeping {sleep_for/60:.1f} min until reset …")
                time.sleep(sleep_for + 1)
                current_token = next(token_cycle) # Switch to the next token
                headers['Authorization'] = f'token {current_token}' # Update header with new token
                continue

            # --- Success / known cases ---
            if r.status_code in (200, 204, 404):
                return r
            else:
                print(f"[WARN] {r.status_code} on {url}")
        except requests.exceptions.RequestException as e:
            print(f"[ERROR] Request failed: {e}")
        # --- backoff ---
        time.sleep(sleep_base * (2 ** attempt) + random.uniform(0, 1))
    return None

def handle_graphql_errors(data, context_info=None):
    """Handles GraphQL errors returned in the response body, including rate limits."""
    if 'errors' in data:
        error_message = f"GraphQL errors for {context_info or 'request'}: {data['errors']}"
        print(f"[GRAPHQL ERROR] {error_message}")
        for error in data['errors']:
            if error.get('type') == 'RATE_LIMITED':
                 print("[RATE-LIMITED] Retrying after a delay...")
                 time.sleep(60) # Wait for a minute before retrying
                 # Note: Token cycling for GraphQL is handled in the calling function (e.g., get_author_association)
                 return 'RATE_LIMITED' # Indicate rate limit error
        return 'ERROR' # Indicate other GraphQL error
    return None # No GraphQL errors

async def get_repo_data(repo_url: str):
    # Make the Request using safe_get
    print(f'Requesting: {repo_url}')
    # Use safe_get which includes token cycling and rate limit handling
    response = safe_get(repo_url)

    if response is None:
        print(f"Error: Failed to retrieve data for {repo_url} after multiple retries.")
        return None

    response.raise_for_status()
    return response.json()


def get_repo_created_at(repo_url: str):
    try:
        task = asyncio.create_task(get_repo_data(repo_url))
        event_loop = asyncio.get_running_loop()
        if event_loop.is_running():
          data = event_loop.run_until_complete(task)
        else:
          data = asyncio.run(task)
        if data is None:
            return None

        created_at = data['created_at']
        print(f"Repo: {repo_url}; Created At: {created_at}")

        if created_at:
            return pd.to_datetime(created_at)
        else:
            raise Exception(f"Error: Could not retrieve createdAt for {repo_url}. Response data: {data}")

    except requests.exceptions.RequestException as e:
        print(f"Error during GitHub API request for {repo_url}: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

SecretNotFoundError: Secret GITHUB_TOKEN_2 does not exist.

## Create a Copy of Pull_Requests & Data Cleaning

1. **Copies DataFrames:** It creates copies of the pull_request and repository DataFrames and assigns them to metrics and repos respectively. This is a good practice to avoid modifying the original loaded data.
2. **Renames Columns:** It renames the 'id' column to 'pr_id' in the metrics DataFrame and to 'repo_id' in the repos DataFrame. This is done to prepare for merging these DataFrames later.
3. **Filters Open Pull Requests:** It removes pull requests that are still open by filtering out rows where the 'closed_at' column has a missing value (NaN).
4. **Converts Timestamps:** It converts the 'created_at' and 'closed_at' columns in the metrics DataFrame to datetime objects. This allows for easier time-based calculations.
5. **Filters Repositories:** It removes repositories from the repos DataFrame that do not have any closed pull requests in the metrics DataFrame.
6. **Gets Repository Creation Dates:** For the remaining repositories, it calls the get_repo_created_at function (defined in a previous cell) to fetch the creation date of each repository from the GitHub API and stores it in a new column 'repo_created_at'.
7. **Filters Repositories with Creation Dates:** It removes repositories where the 'repo_created_at' could not be retrieved.

In [28]:
import os, time, random, requests, pandas as pd
from datetime import datetime
from itertools import cycle # Import cycle

# Copy raw data
metrics = pull_request.copy()
repos = repository.copy()

# === BASIC CLEANUP ===
metrics = metrics.rename(columns={'id': 'pr_id'})
repos = repos.rename(columns={'id': 'repo_id'})

print(f"Total PRs before filtering: {len(metrics):,}")
metrics = metrics[metrics['closed_at'].notna()]
print(f"Closed PRs retained: {len(metrics):,}")

metrics['created_at'] = pd.to_datetime(metrics['created_at'], errors='coerce')
metrics['closed_at'] = pd.to_datetime(metrics['closed_at'], errors='coerce')

print(f"Total repos before filtering: {len(repos):,}")
repos = repos[repos['repo_id'].isin(metrics['repo_id'])]
print(f"Repos with ≥1 PR: {len(repos):,}")

# === ADD: SMALL-SUBSET TEST MODE ===
# Toggle to True for testing on limited data before full run
TEST_MODE = True
if TEST_MODE:
    # Select the next 500 repos (from index 500 to 1000)
    start_index = 500
    end_index = 1000
    if end_index > len(repos):
        end_index = len(repos)
        print(f"[TEST MODE] Adjusting end index to {end_index} as it exceeds the number of repos.")

    test_repo_ids = repos['repo_id'].iloc[start_index:end_index]

    metrics = metrics[metrics['repo_id'].isin(test_repo_ids)]
    repos = repos[repos['repo_id'].isin(test_repo_ids)]
    print(f"[TEST MODE] Restricting to {len(repos)} repos and {len(metrics)} PRs (indices {start_index} to {end_index-1})")


# The safe_get function and token setup have been moved to the cell above
# to ensure they are defined before being called.

Total PRs before filtering: 932,791
Closed PRs retained: 859,927
Total repos before filtering: 116,211
Repos with ≥1 PR: 91,526
[TEST MODE] Restricting to 500 repos and 3239 PRs (indices 500 to 999)


In [29]:
display(metrics.head())
display(repos.head())

,pr_id,number,title,body,agent,user_id,user,state,created_at,closed_at,merged_at,repo_id,repo_url,html_url
2,3264042289,1600,Add Evals frontend implementation plan and HTM...,\nCreate comprehensive implementation plan for...,Claude_Code,6766889,justicart,closed,2025-07-25 18:26:15+00:00,2025-07-25 23:19:14+00:00,None,9.267118e+08,https://api.github.com/repos/bolt-foundry/bolt...,https://github.com/bolt-foundry/bolt-foundry/p...
3,3264042318,1601,Add 4 new BfDs components for Evals interface ...,\nPhase 1 component creation for the Evals fro...,Claude_Code,6766889,justicart,closed,2025-07-25 18:26:16+00:00,2025-07-25 23:19:11+00:00,None,9.267118e+08,https://api.github.com/repos/bolt-foundry/bolt...,https://github.com/bolt-foundry/bolt-foundry/p...
5,3264372403,1,Comprehensive LMSR Markets System with Weekly ...,## Summary\n🚀 **Major platform upgrade**: Comp...,Claude_Code,62402155,derspotter,closed,2025-07-25 20:59:39+00:00,2025-07-28 12:01:05+00:00,2025-07-28T12:01:05Z,9.249577e+08,https://api.github.com/repos/derspotter/intell...,https://github.com/derspotter/intellacc.com/pu...
12,3264552777,54,Issue #47: Complete End-to-End Integration Tes...,## Summary\n\nThis PR implements comprehensive...,Claude_Code,1257915,datablogin,closed,2025-07-25 22:39:02+00:00,2025-07-25 23:29:29+00:00,2025-07-25T23:29:29Z,1.022864e+09,https://api.github.com/repos/datablogin/analyt...,https://github.com/datablogin/analytics-backen...
18,3264677066,1606,Document codebot networking implementation str...,\nAdd implementation memo outlining the curren...,Claude_Code,448694,randallb,closed,2025-07-25 23:52:25+00:00,2025-07-26 00:28:14+00:00,2025-07-26T00:28:14Z,9.267118e+08,https://api.github.com/repos/bolt-foundry/bolt...,https://github.com/bolt-foundry/bolt-foundry/p...


,repo_id,url,license,full_name,language,forks,stars
550,7346775,https://api.github.com/repos/bendrucker/bendru...,NOASSERTION,bendrucker/bendrucker.me,Astro,0.0,0.0
552,1025441210,https://api.github.com/repos/berrylands/modal-...,None,berrylands/modal-meigen-multitalk,Python,0.0,1.0
553,952643099,https://api.github.com/repos/bigdra50/Material...,MIT,bigdra50/MaterialCombiner,C#,0.0,0.0
554,968101794,https://api.github.com/repos/binary-install/bi...,MIT,binary-install/binstaller,Go,1.0,4.0
555,875493470,https://api.github.com/repos/bitsacco/os,MIT,bitsacco/os,TypeScript,4.0,9.0


Mount drive to save Dataset for all of our access

Save initial processed Dataset (no need to re-run this every time)

In [ ]:
metrics.to_pickle("/content/drive/MyDrive/AIDev_shared/metrics_cleaned.pkl")
repos.to_pickle("/content/drive/MyDrive/AIDev_shared/repos_cleaned.pkl")

In [ ]:
#FOR TESTING
metrics.to_pickle("/content/drive/MyDrive/AIDev_shared/metrics_TEST.pkl")
repos.to_pickle("/content/drive/MyDrive/AIDev_shared/repos_TEST.pkl")

Reload dataset (this can be ran every time)

In [ ]:
import pandas as pd
metrics = pd.read_pickle("/content/drive/MyDrive/AIDev_shared/metrics_cleaned.pkl")
repos = pd.read_pickle("/content/drive/MyDrive/AIDev_shared/repos_cleaned.pkl")
print("✅ Loaded shared cached cleaned dataset")

In [ ]:
#FOR TESTING
import pandas as pd
metrics = pd.read_pickle("/content/drive/MyDrive/AIDev_shared/metrics_TEST.pkl")
repos = pd.read_pickle("/content/drive/MyDrive/AIDev_shared/repos_TEST.pkl")
print("✅ Loaded TEST shared cached cleaned dataset")

If we decide to update this dataset again or for frequent updates-- use this to prevent overwriting

In [ ]:
from datetime import date
tag = date.today().isoformat()
metrics.to_pickle(f"/content/drive/MyDrive/AIDev_shared/metrics_{tag}.pkl")
repos.to_pickle(f"/content/drive/MyDrive/AIDev_shared/metrics_{tag}.pkl")

# **Third**, Gather the covariant variables

## PR Variables

1. **additions:** The # of added LOC by a PR
2. **deletions:** The # of deleted LOC by a PR
3. **prSize:** The total number of added and deleted LOC by a PR (additions + deletions)
4. **purpose:** The purpose of a PR, i.e., bug, document, and feature. Simple keyword search in the title/body ('fix', 'bug', 'doc', …).
5. **changedFiles:** The # of files changed by a PR
6. **commitsTotalCount:** The # of commits involved in a PR
7. **bodyLength**: Length of the PR body (in characters).
8. **prExperience:** The # of prior PRs that were submitted by the PR author (author’s prior PR count). Query the author’s PR history in the same repo and count PRs created before the current one.
9. **commentsTotalCount:** The # of comments left on a PR
10. **authorComments:** The # of comments left by the PR author
11. **reviewersComments:** The # of comments left by the reviewers who participate in the disucssion
12. **reviewersTotalCount:** The # of developers who participate in the discussion (excluding author).
13. **repoAge:** Time interval between the repository creation time and PR creation time in days.
14. **isMember:** Whether or not the author is a member or outside collaborator (True/False).
15. **state**: State of the pull request (MERGED or CLOSED).
16. **reviewTime**: Time taken to review the PR (in hours, floating point, no rounding).

## Project variables

17. **repoLanguage:** Programming language of the repository (e.g., Python, PHP, TypeScript, Vue). *[I'm assuming its the top language as there is only one]*
18. **forkCount:** The # of forks that a repository has
19. **stargazerCount:** The # of stargazers that a repository has.

## Treatment variables

20. **With Copilot for PRs:** Whether or not a PR is generated by Copilot for PRs (binary)

## Outcome variables

21. **Review time (reviewTime):** Time interval between the PR creation time and closed time in hours
22. **Is merged (state):** Whether or not a PR is merged (binary)



# PR Variables

1. **additions:** The # of added LOC by a PR
2. **deletions:** The # of deleted LOC by a PR
3. **prSize:** The total number of added and deleted LOC by a PR (additions + deletions)

**Xiao 2024:**

In the notebook (e.g., CollectCopilot4prs.ipynb), the `additions` and `deletions` values are extracted directly from the GitHub API response for each PR: `pr['node']['additions']` and `pr['node']['deletions']`. The GraphQL query for PRs includes the fields, so the value is as reported by GitHub. `prSize = additions + deletions`


**Our Approach:**

In the `pr_commit_details` DataFrame, we use the `additions` and `deletions` fields. We sum them for `prSize`. Alternatively, the dataset also has`changes` which represents prSize but we chose to perform the sum ourselves.


In [ ]:
# Make sure we don't crash because the columns already exist (reentrant code)
metrics = metrics.drop(columns=['additions', 'deletions', 'prSize'], errors='ignore')

print(f"Number of PRs before adding LOC metrics: {len(metrics):,}")

# Get the sums of the columns we are interested in
pr_commit_LOC = (pr_commit_details.groupby(['pr_id'])
                                  .sum(['additions', 'deletions', 'changes'])
                                  .reset_index())

# Rename the sum columns to what we want
pr_commit_LOC = (pr_commit_LOC.rename(columns={'changes': 'prSize'}))

# Drop the extraneous columns
pr_commit_LOC = pr_commit_LOC.drop(columns=['commit_stats_total', 'commit_stats_additions', 'commit_stats_deletions'])

# Merge the Dataframes with a left join
metrics = pd.merge(metrics, pr_commit_LOC, left_on='pr_id', right_on='pr_id', how='left')

# Garbage collect the temporary Dataframe
pr_commit_LOC = None

# Fill N/A values with defaults
metrics['additions'] = metrics['additions'].fillna(0).astype(int)
metrics['deletions'] = metrics['deletions'].fillna(0).astype(int)
metrics['prSize'] = metrics['prSize'].fillna(0).astype(int)

print(f"Number of PRs after adding LOC metrics: {len(metrics):,}")

4. **purpose:** The purpose of a PR, i.e., bug, document, and feature. Simple keyword search in the title/body ('fix', 'bug', 'doc', …).

**Xiao 2024:**

In `CollectCopilot4prs.ipynb`, the code uses `np.select` with conditions based on the PR's title and body content to assign "Bug", "Document", or "Feature" as the purpose. This is a simple rule-based classification:

- If the title/body contains keywords for bugs (e.g., "fix", "bug"), it's labeled "Bug".
- If it contains documentation keywords (e.g., "doc"), it's labeled "Document".
- Otherwise, it's labeled "Feature".


**Our approach:**

The `title` and `body` columns are part of the initial dataset that was loaded into the pull_request (`all_pull_request.parquet`) DataFrame.


In [ ]:
# Make sure we don't crash because the columns already exist (reentrant code)
metrics = metrics.drop(columns=['purpose'], errors='ignore')

print(f"Number of PRs before calculating purpose: {len(metrics):,}")

# Combine title and body for keyword search, handling potential None values
metrics['title_body'] = metrics['title'].fillna('') + ' ' + metrics['body'].fillna('')

# Define conditions and choices for np.select
conditions = [
    metrics['title_body'].str.contains('fix|bug', case=False, na=False),
    metrics['title_body'].str.contains('doc', case=False, na=False)
]
choices = ['fix', 'doc']

# Apply np.select to determine purpose
metrics['purpose'] = np.select(conditions, choices, default='feat')

# Drop the temporary combined column
metrics = metrics.drop(columns=['title_body'])

print(f"Number of PRs after calculating purpose: {len(metrics):,}")

5. **changedFiles:** The # of files changed by a PR

**Xiao 2024:**

The `changedFiles` field is extracted directly from the GitHub API for each pull request. In the code (e.g., `in CollectCopilot4prs.ipynb`), it is accessed as: `pr['node']['changedFiles']`.


**Our Approach:**


This variable is calculated from the `pr_commit_details` DataFrame, steps include:

- Identify the PR ID: The code uses `groupby(['pr_id', 'filename'])` which implicitly identifies each PR by its `pr_id`.
- Locate the file-level change records: It operates on the `pr_commit_details` DataFrame, which contains the file-level change records.
- Collect all rows belonging to the same PR: The `groupby(['pr_id', 'filename'])` operation groups all rows for a specific PR together.
- Count the number of unique filenames for each `pr_id` across all its commits.

In [ ]:
# Make sure we don't crash because the columns already exist (reentrant code)
metrics = metrics.drop(columns=['changedFiles'], errors='ignore')

print(f"Number of PRs before adding changedFiles: {len(metrics):,}")

# Count the number of Files changed and change the column name to what we want
pr_files_changed = (pr_commit_details.groupby(['pr_id', 'filename'])
                                     .size()
                                     .groupby(['pr_id'])
                                     .size()
                                     .reset_index(name='changedFiles'))

# Merge the Dataframes with a left join
metrics = pd.merge(metrics, pr_files_changed, left_on='pr_id', right_on='pr_id', how='left')

# Garbage Collect the temporary Dataframe
pr_files_changed = None

# Fill N/A values with defaults
metrics['changedFiles'] = metrics['changedFiles'].fillna(0).astype(int)

print(f"Number of PRs after adding changedFiles: {len(metrics):,}")

6. **commitsTotalCount:** The # of commits involved in a PR


**Xiao, 2024:**

Fetched from GitHub’s GraphQL API by querying the PR’s `commits { totalCount }` field.

**Our Approach:**

The `pr_commit_details` table contains a `sha` column (the commit hash) and a `pr_id` column that links each commit to its pull request. Count every distinct `sha` in the entire table. Group by `pr_id` and count distinct `sha` values for each group.


In [ ]:
# Make sure we don't crash because the columns already exist (reentrant code)
metrics = metrics.drop(columns=['commitsTotalCount'], errors='ignore')

print(f"Number of PRs before adding commitsTotalCount: {len(metrics):,}")

# Calculate the number of unique commits for each Pull Request from pr_commit_details
# Group by pr_id and count the number of unique sha values
pr_commits_count = pr_commit_details.groupby('pr_id')['sha'].nunique().reset_index(name='commitsTotalCount')

# Merge the Dataframes using a left join
metrics = pd.merge(metrics, pr_commits_count, left_on='pr_id', right_on='pr_id', how='left')

# Garbage Collect the temporary Dataframe
pr_commits_count = None

# Fill N/A values with defaults
metrics['commitsTotalCount'] = metrics['commitsTotalCount'].fillna(0).astype(int)

print(f"Number of PRs after adding commitsTotalCount: {len(metrics):,}")

7. **bodyLength:** The length of a PR description

**Xiao, 2024:**

Query each PR with `pullRequest { body }`. Take the returned text and compute its character count (e.g., len(body)). Record that count as the value of description length.



**Our Approach:**

In the `pull_request` DataFrame, calculate the character length of the `body` column.


In [ ]:
# Make sure we don't crash because the columns already exist (reentrant code)
metrics = metrics.drop(columns=['bodyLength'], errors='ignore')

print(f"Number of PRs before adding bodyLength: {len(metrics):,}")

# Get the Length of the Body of the Pull Request
metrics['bodyLength'] = metrics['body'].str.len()

print(f"Number of PRs after adding bodyLength: {len(metrics):,}")

8. **prExperience:** The # of prior PRs that were submitted by the PR author (author’s prior PR count). Query the author’s PR history in the same repo and count PRs created before the current one.


**Xiao 2024:**

For every pull request the study queries GitHub’s GraphQL API and extracts the author.login (or author.id) and the repository identifier (repository.id). Using the same API they request all pull requests belonging to the same repository.id whose author.login matches the author of the target PR. Each of these PRs includes its createdAt timestamp. The list is filtered to keep only those PRs whose createdAt value is earlier than the createdAt timestamp of the target PR. The number of remaining PRs is taken as an integer count.




**Our Approach:**

- Extract the author's login from the `user` column.
- Sorts the metrics DataFrame by `repo_id`, `author_login`, and the PR creation time (`created_at`).
- Groups the sorted DataFrame by both `repo_id` and `author_login`.
- Within each group (for each unique author in each unique repository), it uses the `.cumcount()` method. `cumcount()` assigns a sequential number starting from 0 to each row within the group based on the current order (which is sorted by `created_at`).




In [ ]:
# Make sure we don't crash because the columns already exist (reentrant code)
metrics = metrics.drop(columns=['prExperience'], errors='ignore')

print(f"Number of PRs before adding prExperience: {len(metrics):,}")

# Extract the author's login from the 'user' column and store it in a new column 'author_login'
metrics['author_login'] = metrics['user'].astype(str).str.strip()

# Drop rows where 'repo_id' or 'created_at' are missing, as these are needed for sorting and calculation
metrics = metrics.dropna(subset=['repo_id', 'created_at'])

# Sort the DataFrame by 'repo_id', 'author_login', and 'created_at' in ascending order--This is crucial for correctly calculating the cumulative count of PRs for each author within each repository.
metrics = metrics.sort_values(['repo_id', 'author_login', 'created_at'])

# Calculate the cumulative count of PRs for each author within each repository.
# The `groupby(['repo_id', 'author_login'])` groups the DataFrame by repository and author.
# The `cumcount()` method then calculates the number of previous PRs for each row within those groups.
metrics['prExperience'] = (
    metrics.groupby(['repo_id', 'author_login'])
           .cumcount()
           .astype('Int64')
)

print(f"Number of PRs after adding bodyLength: {len(metrics):,}")

9. **commentsTotalCount:** The # of comments left on a PR



**Xiao 2024:**

`commentsTotalCount` is obtained directly from the GitHub GraphQL API. For each pull request it queries the PR’s `comments` connection and records the `totalCount` field, which is the number of comment objects attached to that PR (including review comments, issue‑style comments, and any other discussion entries).



**Our Approach:**

Group the `pr_comments` DataFrame by `pr_id` (Pull Request ID). Each row in `pr_comments` represents a single comment.
Count the number of rows within each group using `.size()`.





In [ ]:
# Make sure we don't crash because the columns already exist (rentrant code)
metrics.drop(columns=['commentsTotalCount'], errors='ignore', inplace=True)

print(f"Number of PRs before adding commentsTotalCount: {len(metrics):,}")

# Count the number of Comments for the Pull Request, name the column what we want.
pr_comments_count = pr_comments.groupby(['pr_id']).size().reset_index(name='commentsTotalCount')

# Merge the Dataframes using a left join
metrics = pd.merge(metrics, pr_comments_count, left_on='pr_id', right_on='pr_id', how='left')

# Garbage Collect the temporary Dataframe
pr_comments_count = None

# Fill N/A values with defaults
metrics['commentsTotalCount'] = metrics['commentsTotalCount'].fillna(0).astype(int)

print(f"Number of PRs after adding commentsTotalCount: {len(metrics):,}")

10. **authorComments:** The # of comments left by the PR author

**Xiao 2024:**

Query each PR’s comment data through the GitHub GraphQL API and then filter the comment list to keep only those whose `author.login` matches the PR author’s login.

**Our Approach:**

Merges `pr_comments` with author information and counting comments where the comment author matches the PR author.

In [ ]:
# Make sure we don't crash because the columns already exist (reentrant code)
metrics = metrics.drop(columns=['authorComments'], errors='ignore')

print(f"Number of PRs before adding authorComments: {len(metrics):,}")

# Filter comments to only include those made by the PR author
# Need to merge with metrics to get the author_id for each pr_comment
author_comments = pd.merge(pr_comments, metrics[['pr_id', 'user_id']], left_on='pr_id', right_on='pr_id', how='left')
author_comments = author_comments[author_comments['user_id_x'] == author_comments['user_id_y']]

# Count the number of author comments per pull request
author_comments_count = author_comments.groupby(['pr_id']).size().reset_index(name='authorComments')

# Merge the Dataframes using a left join
metrics = pd.merge(metrics, author_comments_count, left_on='pr_id', right_on='pr_id', how='left')

# Garbage Collect the temporary Dataframes
author_comments = None
author_comments_count = None

# Fill N/A values with defaults
metrics['authorComments'] = metrics['authorComments'].fillna(0).astype(int)

print(f"Number of PRs after adding authorComments: {len(metrics):,}")

11. **reviewersComments:** The # of comments left by the reviewers who participate in the disucssion



**Xiao 2024:**

Querying the `comments` edge of each PR via GraphQL (or the REST endpoint `GET /repos/{owner}/{repo}/issues/{pull_number}/comments`). Filtering out comments whose user.login equals the PR author’s login. Counting the remaining comments – that number is the reviewerComments value.


**Our Approach:**

Merges `pr_comments` with author information and counting comments where the comment author does not match the PR author.



In [ ]:
# Make sure we don't crash because the columns already exist (reentrant code)
metrics = metrics.drop(columns=['reviewersComments'], errors='ignore')

print(f"Number of PRs before adding reviewersComments: {len(metrics):,}")

# Filter comments to exclude those made by the PR author
# Need to merge with metrics to get the author_id for each pr_comment
reviewer_comments = pd.merge(pr_comments, metrics[['pr_id', 'user_id']], left_on='pr_id', right_on='pr_id', how='left')
reviewer_comments = reviewer_comments[reviewer_comments['user_id_x'] != reviewer_comments['user_id_y']]

# Count the number of reviewer comments per pull request
reviewer_comments_count = reviewer_comments.groupby(['pr_id']).size().reset_index(name='reviewersComments')

# Merge the Dataframes using a left join
metrics = pd.merge(metrics, reviewer_comments_count, left_on='pr_id', right_on='pr_id', how='left')

# Garbage Collect the temporary Dataframes
reviewer_comments = None
reviewer_comments_count = None

# Fill N/A values with defaults
metrics['reviewersComments'] = metrics['reviewersComments'].fillna(0).astype(int)

print(f"Number of PRs after adding reviewersComments: {len(metrics):,}")

12. **reviewersTotalCount:** The # of developers who participate in the discussion (excluding author).

**Xiao 2024:**

Calling the GraphQL endpoint for each PR,
Requesting the `reviewers` (or `reviewRequests`) connection,
Reading the `totalCount` field,
Verifying that the author’s login is excluded (or simply using the `totalCount` as GitHub already excludes the author in the `reviewers` connection).

**Our Approach:**

Calculates `reviewersTotalCount` by summing the counts of unique reviewers from reviews (`reviewers`) for each pull request.



In [35]:
# Ensure we don't crash if columns already exist (reentrant code)
metrics = metrics.drop(columns=['reviewersTotalCount'], errors='ignore')

print(f"Number of PRs before adding reviewersTotalCount: {len(metrics):,}")

# Create a copy of PR reviews with only the necessary columns
pr_reviews_temp = pr_reviews.copy().drop(columns=['id', 'state', 'submitted_at', 'body', 'user_type'], errors='ignore')

# Filter reviews to exclude those made by the PR author
# Merge with metrics to get the author_id for each pr_comment
pr_reviews_temp = pd.merge(pr_reviews_temp, metrics[['pr_id', 'user']], left_on='pr_id', right_on='pr_id', how='left')
pr_reviews_temp = pr_reviews_temp[pr_reviews_temp['user_x'] != pr_reviews_temp['user_y']]

# Count the number of unique non-author users who left review comments per pull request
# Group by 'pr_id' to count per pull request
pr_reviews_temp = pr_reviews_temp.groupby(['pr_id'])['user_x'].nunique().reset_index(name='reviewersTotalCount')

# Merge the Dataframes using a left join
metrics = pd.merge(metrics, pr_reviews_temp, left_on='pr_id', right_on='pr_id', how='left')

# Garbage Collect the temporary Dataframes
pr_reviews_temp = None

# Fill N/A values with defaults (for PRs with no reviews/comments)
metrics['reviewersTotalCount'] = metrics['reviewersTotalCount'].fillna(0).astype(int)

print(f"Number of PRs after adding reviewersTotalCount: {len(metrics):,}")


# Display unique values and their counts for 'reviewersTotalCount'
print("\nUnique values and counts for 'reviewersTotalCount':")
display(metrics['reviewersTotalCount'].value_counts(dropna=False).sort_index())

Number of PRs before adding reviewersTotalCount: 3,239
Number of PRs after adding reviewersTotalCount: 3,239

Unique values and counts for 'reviewersTotalCount':


,count
reviewersTotalCount,
0,3002
1,111
2,61
3,57
4,8


13. **repoAge:** Time interval between the repository creation time and PR creation time in days.


**Xiao 2024:**

- Retrieve the creation timestamp of the repository (the time the repo was first created on GitHub).
- Retrieve the creation timestamp of the pull request under study.
- Compute the time interval between these two timestamps in days



**Our Approach:**


Merging `pr_review_comments` and `pr_reviews` DataFrames with our `metrics` to link comments/reviews to PR authors.
Filtering these merged DataFrames to exclude rows where the comment/review user_id matches the PR author's user_id.
Grouping the filtered data by `pr_id`.
Calculating the count of distinct user_id values (`.nunique()`) within each `pr_id` group for both filtered DataFrames.
Merging these unique counts and summing them per `pr_id` to get the total unique non-author participants across review comments and reviews.



In [ ]:
# Make sure we don't crash because the columns already exist (reentrant code)
metrics = metrics.drop(columns=['repoAge'], errors='ignore')

print(f"Number of PRs before adding repoAge: {len(metrics):,}")

# Copy the Repository dataframe
repos_temp = repos.copy()

# Calculate repo_created_at for the repos_temp DataFrame
repos_temp['repo_created_at'] = repos_temp.apply(lambda row: get_repo_created_at(row['url']), axis=1)

# Drop repos where repo_created_at could not be retrieved
repos_temp = repos_temp.dropna(subset=['repo_created_at'])


# Merge the Dataframes using a left join to bring repo_created_at into metrics
# Ensure we only merge the necessary columns to avoid duplicates like repo_created_at_x
metrics = pd.merge(metrics, repos_temp[['repo_id', 'repo_created_at']], on='repo_id', how='left')

# Garbage Collect the temporary dataframe
repos_temp = None

# Drop from Metrics any Repo without a repo created date after the merge
# This might be redundant if merged_at is already checked, but good for safety
metrics = metrics.dropna(subset=['repo_created_at'])

# Calculate the Repo Age in Days (created_at - repo_created_at)
# Ensure 'created_at' and 'repo_created_at' are in datetime format before calculation
metrics['created_at'] = pd.to_datetime(metrics['created_at'], errors='coerce')
metrics['repo_created_at'] = pd.to_datetime(metrics['repo_created_at'], errors='coerce')

metrics = metrics.assign(repoAge=lambda x: (x['created_at'] - x['repo_created_at']).dt.days)

# Drop the unnecessary Repo Created At column after calculating repoAge
metrics = metrics.drop(columns=['repo_created_at'], errors='ignore')

# Fill N/A values for repoAge with defaults
metrics['repoAge'] = metrics['repoAge'].fillna(0).astype(int)

print(f"Number of PRs after adding repoAge: {len(metrics):,}")

14. **isMember:** Whether or not the author is a member or outside collaborator (True/False).


**Xiao 2024:**

- For each PR the study calls GitHub’s GraphQL API and retrieves the author’s association with the repository (the `authorAssociation` field).
- If the returned association is `MEMBER` or `OWNER`, the flag is set to 1; otherwise (e.g., `CONTRIBUTOR`, `NONE`, or an external collaborator) it is set to 0.


**Our Approach:**

Followed the same approach as Xiao 2024.

In [ ]:
# Make sure we don't crash because the columns already exist (reentrant code)
metrics = metrics.drop(columns=['isMember'], errors='ignore')

print(f"Number of PRs before adding isMember: {len(metrics):,}")

def get_author_association(repo_owner: str, repo_name: str, pr_number: int) -> str:
    """Fetches the authorAssociation for a PR using GitHub GraphQL API, cycling through tokens."""
    # Use the token_cycle defined in cell 818abba5
    current_token = next(token_cycle) # Get the next token in the cycle
    query = """
    query($owner: String!, $repo: String!, $pr: Int!) {
      repository(owner: $owner, name: $repo) {
        pullRequest(number: $pr) {
          authorAssociation
        }
      }
    }
    """
    variables = {
        "owner": repo_owner,
        "repo": repo_name,
        "pr": pr_number
    }
    headers = {
        "Authorization": f"Bearer {current_token}", # Use the current token
        "Content-Type": "application/json"
    }

    # Implement basic rate limit handling and retry
    for attempt in range(3): # Retry up to 3 times
        try:
            response = requests.post("https://api.github.com/graphql", json={'query': query, 'variables': variables}, headers=headers)
            response.raise_for_status() # Raise an HTTPError for bad responses

            data = response.json()

            # Handle GraphQL errors using the centralized function
            error_status = handle_graphql_errors(data, context_info=f"PR {pr_number} in {repo_owner}/{repo_name}")

            if error_status == 'RATE_LIMITED':
                # Switch to the next token before retrying due to rate limit
                current_token = next(token_cycle)
                headers['Authorization'] = f'Bearer {current_token}'
                continue # Continue to the next retry attempt

            if error_status == 'ERROR':
                 # For other GraphQL errors, break the retry loop
                 break

            # If no GraphQL errors and no HTTP errors, extract the data
            association = data['data']['repository']['pullRequest']['authorAssociation']
            return association

        except requests.exceptions.RequestException as e:
            print(f"Request error for PR {pr_number} in {repo_owner}/{repo_name}: {e}")
            time.sleep(5 * (attempt + 1)) # Exponential backoff
        except Exception as e:
            print(f"An unexpected error occurred for PR {pr_number} in {repo_owner}/{repo_name}: {e}")
            break # Don't retry for unexpected errors

    return None # Return None if all retries fail


# Prepare data: Ensure 'repo_url' and 'number' are available in metrics
# You might need to merge with the original pull_request DataFrame if these columns were dropped
if 'repo_url' not in metrics.columns or 'number' not in metrics.columns:
     print("Warning: 'repo_url' or 'number' not found in metrics. Merging with original pull_request data.")
     # Assuming original pull_request is available
     metrics = pd.merge(metrics, pull_request[['id', 'repo_url', 'number']], left_on='pr_id', right_on='id', how='left', suffixes=('', '_original'))
     metrics = metrics.drop(columns=['id_original'], errors='ignore') # Drop duplicate id column


# Extract repo owner and name from repo_url
def extract_repo_details(repo_url):
    try:
        parts = repo_url.split('/')
        owner = parts[-2]
        name = parts[-1]
        return owner, name
    except Exception:
        return None, None

metrics[['repo_owner', 'repo_name']] = metrics['repo_url'].apply(lambda url: pd.Series(extract_repo_details(url)))


# Apply the function to fetch author association for each PR
# This can be slow due to API calls. Consider applying to a subset for testing.
# Ensure TEST_MODE is handled if you want to limit API calls
if 'TEST_MODE' in globals() and TEST_MODE:
    print("Running in TEST_MODE, applying to a subset.")
    # Apply to the current subset of metrics
    metrics['authorAssociation'] = metrics.apply(
        lambda row: get_author_association(row['repo_owner'], row['repo_name'], row['number']),
        axis=1
    )
else:
     print("Running on the full dataset.")
     metrics['authorAssociation'] = metrics.apply(
        lambda row: get_author_association(row['repo_owner'], row['repo_name'], row['number']),
        axis=1
    )


# Map authorAssociation to isMember (1 for MEMBER/OWNER, 0 otherwise)
metrics['isMember'] = metrics['authorAssociation'].apply(
    lambda x: 1 if x in ['MEMBER', 'OWNER'] else (0 if x is not None else None) # Handle None from API errors
)

# Drop temporary columns used for API calls if they are not needed for further analysis
metrics = metrics.drop(columns=['repo_owner', 'repo_name', 'authorAssociation'], errors='ignore')


print(f"Number of PRs after adding isMember: {len(metrics):,}")

print("✅ Calculated isMember using GraphQL API.")
display(metrics[['isMember']].head())
display(metrics['isMember'].value_counts(dropna=False))

### **GraphQL Errors Example for isMember**

GraphQL errors for PR ...: [{'type': 'NOT_FOUND', ... 'message': "Could not resolve to a Repository with the name 'owner/repo'."}]: These are error messages from the GitHub GraphQL API. The NOT_FOUND type means that the API could not find the specific resource you were requesting. In these cases, the message "Could not resolve to a Repository with the name 'owner/repo'" indicates that the repository specified in the API call (e.g., 'vals-ai/vals-sdk', 'ImSingingInTheRain/gpai-assessment') could not be found on GitHub. This could be because the repository was renamed, deleted, or is private and your token doesn't have access.


GraphQL errors for PR ...: [{'type': 'NOT_FOUND', ... 'message': 'Could not resolve to a PullRequest with the number of XX.'}]: Similarly, this error indicates that the specific pull request number within the given repository could not be found. This might happen if the PR was closed and potentially deleted, or the number is incorrect for that repository.

15. **state**: State of the pull request (MERGED or CLOSED).


**Xiao 2024:**

In GitHub GraphQL (or REST) API, the response includes a field called state (or, equivalently, a Boolean merged flag). The value of that field can be one of three mutually‑exclusive statuses:
- MERGED (or merged = true)	The PR was successfully merged into the target branch.
- CLOSED (or merged = false & state = CLOSED)	The PR was closed without being merged.
- OPEN (or state = OPEN)	The PR was still open at the time the data were collected.


**Our Approach:**

If the `merged_at` column for a pull request has a value (i.e., it's not `null`), it means the pull request was merged, and the state is set to `MERGED`.
If the `merged_at` column is `null`, it means the pull request was closed without being merged, and the state is set to `CLOSED`.




In [ ]:
# Make sure we don't crash because the columns already exist (reentrant code)
metrics = metrics.drop(columns=['state'], errors='ignore')

print(f"Number of PRs before adding state: {len(metrics):,}")

# Set the State to MERGED or CLOSED
metrics['state'] = metrics['merged_at'].apply(lambda x: 'MERGED' if x is not None else 'CLOSED')

print(f"Number of PRs after adding state: {len(metrics):,}")

16. **reviewTime**: Time taken to review the PR (in hours, floating point, no rounding).

**Xiao 2024:**

`reviewtime` (in hours) = (PR Closed Timestamp - PR Creation Timestamp).


**Our Approach:**

The difference between the `closed_at` and `created_at` timestamps and converting that duration into hours.




In [ ]:
# Make sure we don't crash because the columns already exist (reentrant code)
metrics = metrics.drop(columns=['reviewTime'], errors='ignore')

print(f"Number of PRs before adding reviewTime: {len(metrics):,}")

# Calculate the Review Time
metrics['reviewTime'] = (metrics['closed_at'] - metrics['created_at']).dt.total_seconds() / 3600

print(f"Number of PRs after adding reviewTime: {len(metrics):,}")

# Project variables

17. **repoLanguage:** Programming language of the repository (e.g., Python, PHP, TypeScript, Vue).

**Xiao 2024:**

The number of stargazers that a repository has

**Our Approach:**

Same approach


18. **forkCount:** The # of forks that a repository has

**Xiao 2024:**


**Our Approach:**


19. **stargazerCount:** The # of stargazers that a repository has.

**Xiao 2024:**


**Our Approach:**





In [ ]:
# Make sure we don't crash because the columns already exist (reentrant code)
metrics = metrics.drop(columns=['repoLanguage', 'forkCount', 'stargazerCount'], errors='ignore')

repos_temp = (repos.copy()
                   .drop(columns=['license', 'repo_url', 'html_url', 'full_name'], errors='ignore')
                   .rename(columns={'language': 'repoLanguage', 'forks': 'forkCount', 'stars': 'stargazerCount'}))

# Group by ID and get the First Record
repos_temp = repos_temp.groupby(['repo_id']).first().reset_index() # Add reset_index() to make repo_id a column again

# Merge the Dataframes using a left join
metrics = pd.merge(metrics, repos_temp, left_on='repo_id', right_on='repo_id', how='left')

# Garbage Collect the temporary Dataframe
repos_temp = None

# Fill N/A values with defaults
metrics['repoLanguage'] = metrics['repoLanguage'].fillna('other')
metrics['forkCount'] = metrics['forkCount'].fillna(0).astype(int)
metrics['stargazerCount'] = metrics['stargazerCount'].fillna(0).astype(int)

# Treatment variables

21. **With Copilot for PRs:** Whether or not a PR is generated by Copilot for PRs (binary)


In [ ]:
# Make sure we don't crash because the columns already exist (rentrant code)
metrics = metrics.drop(columns=['isGeneratedByCopliot'], errors='ignore')

metrics['isGeneratedByCopilot'] = metrics['agent'].apply(lambda x: True if x == 'Copilot' else False) # Corrected column name and capitalization

# Outcome variables

22. **Review time (reviewTime):** Time interval between the PR creation time and closed time in hours


In [ ]:
# Make sure we don't crash because the columns already exist (rentrant code)
metrics = metrics.drop(columns=['reviewTime'], errors='ignore')

# Calculate review time in hours, handling potential NaT values
metrics = metrics.assign(reviewTime=lambda x: (x['closed_at'] - x['created_at']).dt.total_seconds() / 3600)

# Fill N/A values with defaults (e.g., for open PRs)
metrics['reviewTime'] = metrics['reviewTime'].fillna(0)

23. **Is merged (state):** Whether or not a PR is merged (binary)


In [ ]:
# Make sure we don't crash because the columns already exist (reentrant code)
metrics = metrics.drop(columns=['isMerged'], errors='ignore')

# If Merged_At is None, the PR was not merged, otherwise it was
metrics['isMerged'] = metrics['merged_at'].apply(lambda x: 0 if x is None else 1)

# Order in CSV (treatment_metrics.csv and control_metrics.csc)

1. **repoLanguage**
2. **forkCount**
3. **stargazerCount**
4. **repoAge**
5. **state**
6. **deletions**
7. **additions**
8. **changedFiles**
9. **commentsTotalCount**
10. **commitsTotalCount**
11. **prExperience**
12. **isMember**
13. **authorComments**
14. **reviewersComments**
15. **reviewersTotalCount**
16. **bodyLength**
17. **prSize**
18. **reviewTime**
19. **purpose**


Export to CSV

In [ ]:
# Define the path to the Google Drive folder
drive_path = "/content/drive/MyDrive/AIDev_shared/"

# Ensure the directory exists (optional, but good practice)
os.makedirs(drive_path, exist_ok=True)

# Define the desired order of columns for the output CSVs
csv_order = ['repoLanguage',
'forkCount',
'stargazerCount',
'repoAge',
'state',
'deletions',
'additions',
'changedFiles',
'commentsTotalCount',
'commitsTotalCount',
'prExperience',
'isMember',
'authorComments',
'reviewersComments',
'reviewersTotalCount',
'bodyLength',
'prSize',
'reviewTime',
'purpose']

# Split the metrics DataFrame into treatment (Copilot) and control (non-Copilot) based on the 'isGeneratedByCopilot' column
try:
    # Use the isGeneratedByCopilot column for splitting
    treatment_metrics_full = metrics[metrics['isGeneratedByCopilot'] == True].copy()
    control_metrics_full = metrics[metrics['isGeneratedByCopilot'] == False].copy()
except KeyError:
    print("Error: 'isGeneratedByCopilot' column not found in metrics DataFrame. Please ensure it's included in previous steps.")
    raise

# Select and reorder columns for the treatment and control DataFrames
treatment_metrics = treatment_metrics_full[csv_order].copy()
control_metrics = control_metrics_full[csv_order].copy()

# Now, drop rows with NaN values from the column-filtered DataFrames
print(f"Number of treatment PRs before dropping NaNs: {len(treatment_metrics):,}")
treatment_metrics.dropna(inplace=True)
print(f"Number of treatment PRs after dropping NaNs: {len(treatment_metrics):,}")

print(f"Number of control PRs before dropping NaNs: {len(control_metrics):,}")
control_metrics.dropna(inplace=True)
print(f"Number of control PRs after dropping NaNs: {len(control_metrics):,}")


# Define the file paths
treatment_file_path = os.path.join(drive_path, "treatment_metrics.csv")
control_file_path = os.path.join(drive_path, "control_metrics.csv")

# Export to CSV
treatment_metrics.to_csv(treatment_file_path, index=False)
control_metrics.to_csv(control_file_path, index=False)

print(f"✅ Exported treatment metrics to: {treatment_file_path}")
print(f"✅ Exported control metrics to: {control_file_path}")

In [ ]:
metrics.head()

# **Fourth**, Bot detection and filtering employed the methodology of Golzadeh et al. (2022)

simple “bot” username suffix check with a comprehensive, manually verified list of 527 bot accounts
* groundtruthbots.csv - a list of bots from Golzadeh et al.


In [ ]:
import pandas as pd
import os

# Define the path to the Google Drive folder
drive_path = "/content/drive/MyDrive/AIDev_shared/"

# Define the file path for the ground truth bots list (now a CSV)
bots_file_path = os.path.join(drive_path, "groundtruthbots.csv")

# Load the ground truth bots list from CSV
try:
    # Assuming the CSV has a header and the usernames are in the first column
    groundtruth_bots_df = pd.read_csv(bots_file_path)
    # Extract the first column and convert to a list, handling potential NaNs
    groundtruth_bots = groundtruth_bots_df.iloc[:, 0].dropna().astype(str).tolist()
    print(f"Loaded {len(groundtruth_bots)} bot usernames from {bots_file_path}")
except FileNotFoundError:
    print(f"Error: {bots_file_path} not found. Please ensure the file exists and contains bot usernames in the first column.")
    groundtruth_bots = [] # Initialize as empty list to avoid errors later
except Exception as e:
    print(f"An error occurred while reading {bots_file_path}: {e}")
    groundtruth_bots = [] # Initialize as empty list


# --- Filter bot-submitted PRs from metrics DataFrame ---

print(f"Number of PRs before bot filtering: {len(metrics):,}")

# Filter out PRs where the author's login ends with 'bot' or is in the ground truth list
# Ensure 'author_login' column exists and is treated as string
if 'author_login' not in metrics.columns:
    print("Warning: 'author_login' column not found in metrics. Skipping PR bot filtering.")
    # You might want to add a step to create 'author_login' here if it's missing
    # For now, we'll skip this filtering step to avoid crashing
    metrics_filtered = metrics.copy()
else:
    metrics_filtered = metrics[
        (~metrics['author_login'].astype(str).str.lower().str.endswith('bot', na=False)) &
        (~metrics['author_login'].astype(str).isin(groundtruth_bots))
    ].copy() # Use .copy() to avoid SettingWithCopyWarning


print(f"Number of PRs after bot filtering: {len(metrics_filtered):,}")


# --- Filter bot comments from pr_comments DataFrame ---

print(f"Number of comments before bot filtering: {len(pr_comments):,}")

# Filter out comments where the user's login ends with 'bot' or is in the ground truth list
# Ensure 'user' column exists and is treated as string
if 'user' not in pr_comments.columns:
     print("Warning: 'user' column not found in pr_comments. Skipping comment bot filtering.")
     # You might want to add a step to create 'user' here if it's missing
     # For now, we'll skip this filtering step to avoid crashing
     pr_comments_filtered = pr_comments.copy()
else:
    pr_comments_filtered = pr_comments[
        (~pr_comments['user'].astype(str).str.lower().str.endswith('bot', na=False)) &
        (~pr_comments['user'].astype(str).isin(groundtruth_bots))
    ].copy() # Use .copy() to avoid SettingWithCopyWarning


print(f"Number of comments after bot filtering: {len(pr_comments_filtered):,}")

# You can now replace the original metrics and pr_comments DataFrames with the filtered ones
metrics = metrics_filtered
pr_comments = pr_comments_filtered

print("Bot filtering applied to metrics and pr_comments DataFrames.")

# **Fifth**, Adoption Trend (RQ1)
(RQ1) To what extent do developers use Copilot for PRs in the code review process?

* Counted occurrences of each marker tag; copilot:summary was the most frequent (13 231 instances).
* Visualised cumulative PRs over time (Fig. 3) and proportion of PRs per repository (Fig. 4).


# **Sixth**, Causal Inference (RQ2)

### Propensity‑Score Estimation
Logistic regression (treatment = Copilot usage) on the 17 covariates.
Estimated each PR’s probability of receiving the treatment (ps).
### Weight Construction
Inverse‑probability weights: 1/ps for treated, 1/(1‑ps) for control.
### Entropy Balancing
Applied the entropy‑balancing algorithm (equivalent to R’s ebalance) to adjust the raw weights so that the weighted means of all covariates matched exactly between groups.
After balancing, absolute mean differences for every covariate were ≤ 0.10 (Fig. 2).
### Outcome Regression
* Review time (continuous): weighted ordinary least squares (lm analogue) with only the treatment indicator. The coefficient gave the Average Treatment Effect on the Treated (ATT) of ‑19.3 h (p ≈ 1.6 × 10⁻¹⁷).
* Merge outcome (binary): weighted logistic regression (glm with logit link). The exponentiated treatment coefficient yielded an odds ratio of 1.57 (95 % CI [1.35, 1.84], p < 0.001).
These two models answer RQ2.1 (review‑time reduction) and RQ2.2 (higher merge likelihood).


# The R Scripts
The main difference between PMW_merge.R and PMW_review.R is:

* PMW_merge.R includes the column isMerged, which indicates whether each pull request was merged (state == "MERGED"). This column is added to the modeling data and used in the analysis.
* PMW_review.R does not include the isMerged column in its modeling data; it focuses only on review-related metrics.
* Otherwise, both scripts process the same input data, use similar covariates, and prepare for causal inference analysis. The inclusion of isMerged in PMW_merge.R allows for analysis related to PR merge status, while PMW_review.R is focused on review characteristics.